In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### KAIST Scene Text Database 전처리기:
1. 데이터 url:http://www.iapr-tc11.org/mediawiki/index.php/KAIST_Scene_Text_Database
1. 압축을 다 풀고 'kaist_dataset'아래에 이미지 데이터와 xml 파일을 두면
1. 각 xml 파일에서 글자하나하나 위치정보를 읽어 원본 이미지에서 글자를 별도의 이미지 파일로 저장하는 작업을 수행한다.
1. 결과물은 char_data 아래에 각 글자명으로 폴더를 만들고 그 아래에 원본파일명.글자명.idx.jpg로 저장
    - char_data/{character}/{img_filename}.{character}.{idx}.jpg
    - character가 .(dot)인 경우에는 {character}값을 dot으로 대체하여 처리
    

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

base_path = 'phd08_output'

def read_charlist(base_path):
    data_filenames = [path.splitext(f)[0] for f in listdir(base_path) if isfile(join(base_path, f)) and f.lower().endswith('.csv')]        
    
    return data_filenames
        
data_filenames = read_charlist(base_path)
data_filenames = data_filenames[:20]

label_idx = {x:idx for idx, x in enumerate(sorted(data_filenames))}

label_idx

{'가': 0,
 '각': 1,
 '간': 2,
 '갇': 3,
 '갈': 4,
 '갉': 5,
 '갊': 6,
 '감': 7,
 '갑': 8,
 '값': 9,
 '갓': 10,
 '갔': 11,
 '강': 12,
 '갖': 13,
 '갗': 14,
 '같': 15,
 '갚': 16,
 '갛': 17,
 '개': 18,
 '객': 19}

In [3]:
import tensorflow as tf

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('eval_dir', '/tmp/phd08_eval',
                           """Directory where to write event logs.""")
tf.app.flags.DEFINE_string('eval_data', 'test',
                           """Either 'test' or 'train_eval'.""")
tf.app.flags.DEFINE_string('checkpoint_dir', '/tmp/phd08_train',
                           """Directory where to read model checkpoints.""")
tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5,
                            """How often to run the eval.""")
tf.app.flags.DEFINE_integer('num_examples', 1000,
                            """Number of examples to run.""")
tf.app.flags.DEFINE_boolean('run_once', False,
                            """Whether to run eval only once.""")


In [13]:
FLAGS.batch_size = 128
FLAGS.eval_interval_secs = 60*60

In [5]:
def get_inputs():
    # Create a queue that produces the filenames to read.
    filenames = [join(base_path, '.'.join([char, 'csv'])) for char in data_filenames]
    filename_queue = tf.train.string_input_producer(filenames)
    line_reader = tf.TextLineReader()
    # Read a whole file from the queue, the first returned value in the tuple is the
    # filename which we are ignoring.
    _, csv_row = line_reader.read(filename_queue)
    
    '''
        Columns:
        FO(1): Font type (B:바다, D:돋움, G:고딕, H1:한양해서, H2:헤드라인, M:명조, N:나무, S:샘물, Y:엽서)
        FS(1): Font size (0:12, 1:13: 2:14)
        CP(1): The number of copies (0:0, 1:1, 2:2)
        RE(1): Resolution (0:200, 1:240, 2:280)
        TH(1): Threshold (0:140, 1:180, 2:220)
        SL(1): Slope(Rotate) (0:-3deg, 1:0deg, 2:3deg)
        HE(1): Height(pixels)
        WD(1): Width(pixels)
        Korean Character(1) (가, 각, 간, ...)
        Image data(The number of columns = Rows X Cols)
        Label(1): (0:가, 1:각, 2:간 ...)

    '''
    record_defaults = [
        [''], # FO
        [''], # FS
        [0],  # CP
        [0],  # RE
        [0],  # TH
        [0],  # SL
        [0],  # HE
        [0],  # WD
        [''],  # Kor char.
    ] + [[0]] * (56*56) + [[0]]

    columns = tf.decode_csv(csv_row, record_defaults=record_defaults)
    character = columns[8]
    image = tf.reshape(tf.stack(columns[9:-1]), [56, 56, 1])
    image = tf.cast(image, tf.float32)
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.per_image_standardization(image)
    label = columns[-1]
    
    '''
    To check valid input
    '''
#     with tf.Session() as sess:
#         coord = tf.train.Coordinator()
#         threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#         fig = plt.figure(figsize=(40,250))
#         for i in range(10):
#             char_value, l_value, image_value = sess.run([character, label, image])
#             print(str(char_value))
#             print(l_value)
#             print(image_value)
#             subplot = fig.add_subplot(50,10,i+1)
#             subplot.set_xlabel(l_value)
#             plt.imshow(image_value)
#         plt.show()            
#         coord.request_stop()
#         coord.join(threads)             

#     print('image', image)
#     print('label', label)
    
    num_preprocess_threads = 1
    min_queue_examples = 256

    images, label_batch = tf.train.batch(
        [image, label],
        batch_size=FLAGS.batch_size,
        num_threads=num_preprocess_threads,
        capacity=min_queue_examples + 3 * FLAGS.batch_size)

#     tf.summary.image('images', images)
#     tf.summary.histogram('label_batch', label_batch)
    return images, tf.reshape(label_batch, [FLAGS.batch_size])

get_inputs()

(<tf.Tensor 'batch:0' shape=(128, 56, 56, 3) dtype=float32>,
 <tf.Tensor 'Reshape_1:0' shape=(128,) dtype=int32>)

In [6]:
NUM_CLASSES = len(label_idx)
NUM_CLASSES

20

In [7]:
import re
import time
from datetime import datetime

TOWER_NAME = 'tower'
NUM_CLASSES = len(label_idx)


def _activation_summary(x):
    """Helper to create summaries for activations.

    Creates a summary that provides a histogram of activations.
    Creates a summary that measures the sparsity of activations.

    Args:
      x: Tensor
    Returns:
      nothing
    """
    # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
    # session. This helps the clarity of presentation on tensorboard.
    tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity',
                      tf.nn.zero_fraction(x))


def _variable_on_cpu(name, shape, initializer):
    """Helper to create a Variable stored on CPU memory.

    Args:
      name: name of the variable
      shape: list of ints
      initializer: initializer for Variable

    Returns:
      Variable Tensor
    """
    with tf.device('/cpu:0'):
        dtype = tf.float32
        var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
    return var


def _variable_with_weight_decay(name, shape, stddev, wd):
    """Helper to create an initialized Variable with weight decay.

    Note that the Variable is initialized with a truncated normal distribution.
    A weight decay is added only if one is specified.

    Args:
      name: name of the variable
      shape: list of ints
      stddev: standard deviation of a truncated Gaussian
      wd: add L2Loss weight decay multiplied by this float. If None, weight
          decay is not added for this Variable.

    Returns:
      Variable Tensor
    """
    dtype = tf.float32
    var = _variable_on_cpu(
        name,
        shape,
        tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var


NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.1       # Initial learning rate.

def _add_loss_summaries(total_loss):
    """Add summaries for losses in CIFAR-10 model.

    Generates moving average for all losses and associated summaries for
    visualizing the performance of the network.

    Args:
        total_loss: Total loss from loss().
    Returns:
        loss_averages_op: op for generating moving averages of losses.
    """
    # Compute the moving average of all individual losses and the total loss.
    loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
    losses = tf.get_collection('losses')
    loss_averages_op = loss_averages.apply(losses + [total_loss])

    # Attach a scalar summary to all individual losses and the total loss; do the
    # same for the averaged version of the losses.
    for l in losses + [total_loss]:
        # Name each loss as '(raw)' and name the moving average version of the loss
        # as the original loss name.
        tf.summary.scalar(l.op.name + ' (raw)', l)
        tf.summary.scalar(l.op.name, loss_averages.average(l))

    return loss_averages_op

def _train(total_loss, global_step):
    """Train CIFAR-10 model.

    Create an optimizer and apply to all trainable variables. Add moving
    average for all trainable variables.

    Args:
    total_loss: Total loss from loss().
    global_step: Integer Variable counting the number of training steps
      processed.
    Returns:
    train_op: op for training.
    """
    # Variables that affect learning rate.
    num_batches_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / FLAGS.batch_size
    decay_steps = int(num_batches_per_epoch * NUM_EPOCHS_PER_DECAY)

    # Decay the learning rate exponentially based on the number of steps.
    lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                  global_step,
                                  decay_steps,
                                  LEARNING_RATE_DECAY_FACTOR,
                                  staircase=True)
    tf.summary.scalar('learning_rate', lr)

    # Generate moving averages of all losses and associated summaries.
    loss_averages_op = _add_loss_summaries(total_loss)

    # Compute gradients.
    with tf.control_dependencies([loss_averages_op]):
        opt = tf.train.GradientDescentOptimizer(lr)
        grads = opt.compute_gradients(total_loss)

    # Apply gradients.
    apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

    # Add histograms for trainable variables.
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)

    # Add histograms for gradients.
    for grad, var in grads:
        if grad is not None:
            tf.summary.histogram(var.op.name + '/gradients', grad)

    # Track the moving averages of all trainable variables.
    variable_averages = tf.train.ExponentialMovingAverage(
      MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
        train_op = tf.no_op(name='train')

    return train_op

def inference(images):
    """Build the CIFAR-10 model.

    Args:
      images: Images returned from distorted_inputs() or inputs().

    Returns:
      Logits.
    """
    # We instantiate all variables using tf.get_variable() instead of
    # tf.Variable() in order to share variables across multiple GPU training runs.
    # If we only ran this model on a single GPU, we could simplify this function
    # by replacing all instances of tf.get_variable() with tf.Variable().
    #
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 3, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1)
        
    # conv1-1
    with tf.variable_scope('conv1-1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(conv1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1_1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1_1)

    # conv1-2
    with tf.variable_scope('conv1-2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1_2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1_2)
    
    
    # pool1
    pool1 = tf.nn.max_pool(conv1_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                      name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2)

    # conv2-1
    with tf.variable_scope('conv2-1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(conv2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2_1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2_1)

    # conv2-2
    with tf.variable_scope('conv2-2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2_2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2_2)
        
    # norm2
    norm2 = tf.nn.lrn(conv2_2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                      name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [FLAGS.batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 384],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        _activation_summary(local3)

    # local4
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_weight_decay('weights', shape=[384, 192],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        _activation_summary(local4)
    print('local3', local3)
    print('local4', local4)

    # linear layer(WX + b),
    # We don't apply softmax here because
    # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
    # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [192, NUM_CLASSES],
                                              stddev=1 / 192.0, wd=0.0)
        biases = _variable_on_cpu('biases', [NUM_CLASSES],
                                  tf.constant_initializer(0.0))
        
        print('weights', weights)
        print('biases', biases)
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        
        print('softmax_linear', softmax_linear)
        _activation_summary(softmax_linear)

    return softmax_linear

def _loss(logits, labels):
    """Add L2Loss to all the trainable variables.

    Add summary for "Loss" and "Loss/avg".
    Args:
      logits: Logits from inference().
      labels: Labels from distorted_inputs or inputs(). 1-D tensor
              of shape [batch_size]

    Returns:
      Loss tensor of type float.
    """
    # Calculate the average cross entropy loss across the batch.
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)

    # The total loss is defined as the cross entropy loss plus all of the weight
    # decay terms (L2 loss).
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


def train():
    """Train CIFAR-10 for a number of steps."""
    with tf.Graph().as_default():
        images, labels = get_inputs()
        
        print('labels.get_shape()', labels.get_shape())
        global_step = tf.contrib.framework.get_or_create_global_step()
        
        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = inference(images)
        
        print('images, logits, labels', images, logits, labels)

        # Calculate loss.
        loss = _loss(logits, labels)

        # Build a Graph that trains the model with one batch of examples and
        # updates the model parameters.
        train_op = _train(loss, global_step)

        class _LoggerHook(tf.train.SessionRunHook):
            """Logs loss and runtime."""

            def begin(self):
                self._step = -1
                self._start_time = time.time()

            def before_run(self, run_context):
                self._step += 1
                return tf.train.SessionRunArgs(loss)  # Asks for loss value.

            def after_run(self, run_context, run_values):
                if self._step % FLAGS.log_frequency == 0:
                    current_time = time.time()
                    duration = current_time - self._start_time
                    self._start_time = current_time

                    loss_value = run_values.results
                    examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
                    sec_per_batch = float(duration / FLAGS.log_frequency)

                    format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                                  'sec/batch)')
                    print(format_str % (datetime.now(), self._step, loss_value,
                                        examples_per_sec, sec_per_batch))

        with tf.train.MonitoredTrainingSession(
                checkpoint_dir=FLAGS.train_dir,
                hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
                       tf.train.NanTensorHook(loss),
                       _LoggerHook()],
                config=tf.ConfigProto(
                    log_device_placement=FLAGS.log_device_placement)) as mon_sess:
            while not mon_sess.should_stop():
                mon_sess.run(train_op)


In [ ]:
import re
import time
from datetime import datetime
import math

TOWER_NAME = 'tower'
NUM_CLASSES = len(label_idx)


def eval_once(saver, summary_writer, top_k_op, top_5_op, summary_op, logits, labels):
    """Run Eval once.

    Args:
      saver: Saver.
      summary_writer: Summary writer.
      top_k_op: Top K op.
      summary_op: Summary op.
    """
    with tf.Session() as sess:
        ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            # Restores from checkpoint
            saver.restore(sess, ckpt.model_checkpoint_path)
            # Assuming model_checkpoint_path looks something like:
            #   /my-favorite-path/cifar10_train/model.ckpt-0,
            # extract global_step from it.
            global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
        else:
            print('No checkpoint file found')
            return

        # Start the queue runners.
        coord = tf.train.Coordinator()
        try:
            threads = []
            for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
                threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                                 start=True))

            num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
            true_count = 0  # Counts the number of correct predictions.
            true_count_top5 = 0  # Counts the number of correct predictions.
            total_sample_count = num_iter * FLAGS.batch_size
            step = 0
            while step < num_iter and not coord.should_stop():
                predictions, predictions_top5, logit_val, label_val = sess.run([top_k_op, top_5_op, logits, labels])
                
                if step % 1000 == 0:
                    import numpy as np
                    
                    pred_label = [np.argmax(logit) for logit in logit_val]
                    print('logit_val', logit_val)
                    print('pred_label, label_val, predictions', pred_label, label_val, predictions)
                    
#                 pred_label = [np.argmax(logit) for logit in logit_val]
#                 print('logit_val', logit_val)
#                 print('pred_label, label_val, predictions', pred_label, label_val, predictions)
                true_count += np.sum(predictions)
                true_count_top5 += np.sum(predictions_top5)
                step += 1

            # Compute precision @ 1.
            precision = true_count / total_sample_count
            precision_top5 = true_count_top5 / total_sample_count
            print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))
            print('%s: precision @ 5 = %.3f' % (datetime.now(), precision_top5))

            summary = tf.Summary()
            summary.ParseFromString(sess.run(summary_op))
            summary.value.add(tag='Precision @ 1', simple_value=precision)
            summary.value.add(tag='Precision @ 5', simple_value=precision_top5)
            summary_writer.add_summary(summary, global_step)
        except Exception as e:  # pylint: disable=broad-except
            coord.request_stop(e)

        coord.request_stop()
        coord.join(threads, stop_grace_period_secs=10)


def evaluate():
    """Eval CIFAR-10 for a number of steps."""
    with tf.Graph().as_default() as g:
        # Get images and labels for CIFAR-10.
#         eval_data = FLAGS.eval_data == 'test'
#         images, labels = cifar10.inputs(eval_data=eval_data)
        images, labels = get_inputs()

        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = inference(images)
        
        # Calculate predictions.
        top_k_op = tf.nn.in_top_k(logits, labels, 1)
        top_5_op = tf.nn.in_top_k(logits, labels, 5)

        # Restore the moving average version of the learned variables for eval.
        variable_averages = tf.train.ExponentialMovingAverage(
            MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        # Build the summary operation based on the TF collection of Summaries.
        summary_op = tf.summary.merge_all()
        summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)
        
        while True:
            eval_once(saver, summary_writer, top_k_op, top_5_op, summary_op, logits, labels)
            if FLAGS.run_once:
                break
            time.sleep(FLAGS.eval_interval_secs)

evaluate()

local3 Tensor("local3/local3:0", shape=(128, 384), dtype=float32)
local4 Tensor("local4/local4:0", shape=(128, 192), dtype=float32)
weights <tf.Variable 'softmax_linear/weights:0' shape=(192, 20) dtype=float32_ref>
biases <tf.Variable 'softmax_linear/biases:0' shape=(20,) dtype=float32_ref>
softmax_linear Tensor("softmax_linear/softmax_linear:0", shape=(128, 20), dtype=float32)
INFO:tensorflow:Restoring parameters from /tmp/phd08_train/model.ckpt-16886
logit_val [[-0.00116221 -0.12091798 -0.04991271 ...,  0.25093821  0.08635263
   0.07991093]
 [-0.00116221 -0.12091798 -0.04991271 ...,  0.25093821  0.08635263
   0.07991093]
 [-0.00116221 -0.12091798 -0.04991271 ...,  0.25093821  0.08635263
   0.07991093]
 ..., 
 [-0.00116221 -0.12091798 -0.04991271 ...,  0.25093821  0.08635263
   0.07991093]
 [-0.00116221 -0.12091798 -0.04991271 ...,  0.25093821  0.08635263
   0.07991093]
 [-0.00116221 -0.12091798 -0.04991271 ...,  0.25093821  0.08635263
   0.07991093]]
pred_label, label_val, prediction

2017-09-29 14:15:51.775464: precision @ 1 = 0.000
2017-09-29 14:15:51.775672: precision @ 5 = 0.000
INFO:tensorflow:Restoring parameters from /tmp/phd08_train/model.ckpt-20033
logit_val [[ 0.0713073   0.04679133 -0.04766726 ...,  0.08666516  0.06077418
  -0.14177787]
 [ 0.0713073   0.04679133 -0.04766726 ...,  0.08666516  0.06077418
  -0.14177787]
 [ 0.0713073   0.04679133 -0.04766726 ...,  0.08666516  0.06077418
  -0.14177787]
 ..., 
 [ 0.0713073   0.04679133 -0.04766726 ...,  0.08666516  0.06077418
  -0.14177787]
 [ 0.0713073   0.04679133 -0.04766726 ...,  0.08666516  0.06077418
  -0.14177787]
 [ 0.0713073   0.04679133 -0.04766726 ...,  0.08666516  0.06077418
  -0.14177787]]
pred_label, label_val, predictions [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17